In [1]:
from google.colab import drive, output
drive.mount("/content/drive")
import os
os.chdir("/content/drive/My Drive/plant_pathology_kaggle")

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


##Loading & Preprocessing

In [2]:
import numpy as np
import pandas as pd

sam_sub = pd.read_csv('sample_submission.csv')
train_csv = pd.read_csv('train.csv')
test_csv = pd.read_csv('test.csv')

def get_class(row): 
  if row['multiple_diseases'] == 1: 
    return 'multiple_diseases' 
  elif row['rust'] == 1: 
    return 'rust' 
  elif row['scab'] == 1: 
    return 'scab' 
  else: 
    return 'healthy' 
    
train_csv["label"] = train_csv.apply(get_class, axis=1) 
#train_csv.head()
temp = pd.factorize(train_csv["label"])
train_csv["label"] = temp[0]

train_csv.head()

,image_id,healthy,multiple_diseases,rust,scab,label
0,Train_0,0,0,0,1,0
1,Train_1,0,1,0,0,1
2,Train_2,1,0,0,0,2
3,Train_3,0,0,1,0,3
4,Train_4,1,0,0,0,2


In [3]:
y = train_csv["label"]

from keras.utils import to_categorical
y = to_categorical(y, 4)

Using TensorFlow backend.


In [0]:
path = '/content/drive/My Drive/plant_pathology_kaggle/images/'
image_list = os.listdir(path)

train_images = train_csv["image_id"]
test_images = test_csv["image_id"]

train_images = np.asarray(train_images) + ".jpg"
test_images = np.asarray(test_images) + ".jpg"

In [5]:
image_height = 150
image_width = 150
channels = 3

import cv2
xtrain = np.zeros((len(y), image_height, image_width, channels))
xtest = np.zeros((len(y), image_height, image_width, channels))

for i in range(len(y)):
  imgtrain = train_images[i]
  imgtest = test_images[i]
  #path = '/content/drive/My Drive/plant_pathology_kaggle/images/' + img
  img = cv2.imread('/content/drive/My Drive/plant_pathology_kaggle/images/'+imgtrain)
  img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
  img = cv2.resize(img, (image_height, image_width))
  xtrain[i] = img
  img = cv2.imread('/content/drive/My Drive/plant_pathology_kaggle/images/'+imgtest)
  img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
  img = cv2.resize(img, (image_height, image_width))
  xtest[i] = img
  if i%50==0:
    print("loaded " + str(i) + " images")

xtrain /= 255.0
xtest /= 255.0

output.eval_js('new Audio("https://ssl.gstatic.com/dictionary/static/sounds/20180430/complete--_us_1.mp3").play()')

loaded 0 images
loaded 50 images
loaded 100 images
loaded 150 images
loaded 200 images
loaded 250 images
loaded 300 images
loaded 350 images
loaded 400 images
loaded 450 images
loaded 500 images
loaded 550 images
loaded 600 images
loaded 650 images
loaded 700 images
loaded 750 images
loaded 800 images
loaded 850 images
loaded 900 images
loaded 950 images
loaded 1000 images
loaded 1050 images
loaded 1100 images
loaded 1150 images
loaded 1200 images
loaded 1250 images
loaded 1300 images
loaded 1350 images
loaded 1400 images
loaded 1450 images
loaded 1500 images
loaded 1550 images
loaded 1600 images
loaded 1650 images
loaded 1700 images
loaded 1750 images
loaded 1800 images


##Upsampling & Augmentation

In [6]:
from imblearn.over_sampling import RandomOverSampler

ros = RandomOverSampler(random_state = 267)

x_res, y_res = ros.fit_resample(xtrain.reshape((-1, image_height * image_width * channels)), y)
x_res = x_res.reshape((-1, image_height, image_width, channels))

x_res.shape, y_res.sum(axis=0)

/usr/local/lib/python3.6/dist-packages/sklearn/externals/six.py:31: FutureWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.neighbors.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)


((2488, 150, 150, 3), array([622, 622, 622, 622]))

In [0]:
from sklearn.model_selection import train_test_split
xtrain, xcv, ytrain, ycv = train_test_split(x_res, y_res, test_size = 0.2, random_state = 267)

train_size = len(ytrain)

In [0]:
from keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(rotation_range = 45, width_shift_range = 0.2, height_shift_range = 0.2,
                             zoom_range = 0.2, horizontal_flip = True, vertical_flip = True, 
                             brightness_range = [0.8, 1.2])

datagen.fit(xtrain)

## Models to try : (without upsampling, first)
i. EfficientNetB7
ii. DenseNet201 - good

iv. ResNet
vi. Inception
###change image dimensions if you want to 
done with MobileNet - not good, b5 - good

tried all, choosing EffNet and DenseNet


In [9]:
!pip install efficientnet

In [10]:
#import efficientnet.keras as efn
from keras.models import Sequential
from keras.applications.densenet import DenseNet201
from keras.layers import Dense, GlobalAveragePooling2D

base = DenseNet201(weights = 'imagenet', include_top=False, 
                          input_shape = (image_height,image_width, channels))

model = Sequential()
model.add(base)
model.add(GlobalAveragePooling2D())
model.add(Dense(4, activation = "softmax"))
model.summary()

74842112/74836368 [==============================] - 1s 0us/step
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
densenet201 (Model)          (None, 4, 4, 1920)        18321984  
_________________________________________________________________
global_average_pooling2d_1 ( (None, 1920)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 4)                 7684      
Total params: 18,329,668
Trainable params: 18,100,612
Non-trainable params: 229,056
_________________________________________________________________


In [11]:
from keras.optimizers import RMSprop
opti = RMSprop(lr = 1e-3)
model.compile(optimizer = opti, loss = "categorical_crossentropy", metrics = ["accuracy"])

from keras.callbacks import ReduceLROnPlateau
learning_rate_reduction = ReduceLROnPlateau(monitor='val_loss', patience=3, 
                                            verbose=1, factor=0.5, min_lr = 1e-10)

from keras.callbacks import ModelCheckpoint
checkpoint = ModelCheckpoint(filepath = "/content/e_us.h5", 
                             save_best_only = True, monitor = "val_loss",
                             verbose = 1)

In [0]:
history = model.fit_generator(datagen.flow(x = xtrain, y = ytrain, batch_size = 32), 
                    epochs = 100, steps_per_epoch = train_size//32, 
                    validation_data = (xcv, ycv), 
                    callbacks = [learning_rate_reduction, checkpoint])

output.eval_js('new Audio("https://ssl.gstatic.com/dictionary/static/sounds/20180430/complete--_us_1.mp3").play()')

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Epoch 1/100
62/62 [==============================] - 124s 2s/step - loss: 1.2548 - acc: 0.5282 - val_loss: 1.4902 - val_acc: 0.2530

Epoch 00001: val_loss improved from inf to 1.49017, saving model to /content/e_us.h5
Epoch 2/100
62/62 [==============================] - 52s 840ms/step - loss: 0.9930 - acc: 0.6615 - val_loss: 2.8634 - val_acc: 0.2530

Epoch 00002: val_loss did not improve from 1.49017
Epoch 3/100
62/62 [==============================] - 52s 841ms/step - loss: 0.7246 - acc: 0.7346 - val_loss: 12.0724 - val_acc: 0.2510

Epoch 00003: val_loss did not improve from 1.49017
Epoch 4/100
62/62 [==============================] - 52s 842ms/step - loss: 0.6552 - acc: 0.7629 - val_loss: 1.8069 - val_acc: 0.2530

Epoch 00004: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.

Epoch 00004: val_loss did not improve from 1.49017
Epoch 5/100
62/62 [===========================

In [0]:
predictions = model.predict(xtest)

sam_sub["healthy"].values[:] = 0
sam_sub["multiple_diseases"].values[:] = 0
sam_sub["rust"].values[:] = 0
sam_sub["scab"].values[:] = 0

for i in range(len(predictions)):
  sam_sub["scab"].values[i] = predictions[i, 0]
  sam_sub["multiple_diseases"].values[i] = predictions[i, 1]
  sam_sub["healthy"].values[i] = predictions[i, 2]
  sam_sub["rust"].values[i] = predictions[i, 3]

sam_sub.to_csv("/content/e_us_100.csv", index = False)
np.save("/content/e_us_100.npy", predictions)
output.eval_js('new Audio("https://ssl.gstatic.com/dictionary/static/sounds/20180430/complete--_us_1.mp3").play()')

##Predictions from best model

In [0]:
from keras.models import load_model
model = load_model("/content/e_us.h5")
print("loaded!")
predictions = model.predict(xtest)
np.save("/content/e_us_best.npy", predictions)

In [0]:
sam_sub["healthy"].values[:] = 0
sam_sub["multiple_diseases"].values[:] = 0
sam_sub["rust"].values[:] = 0
sam_sub["scab"].values[:] = 0

for i in range(len(predictions)):
  sam_sub["scab"].values[i] = predictions[i, 0]
  sam_sub["multiple_diseases"].values[i] = predictions[i, 1]
  sam_sub["healthy"].values[i] = predictions[i, 2]
  sam_sub["rust"].values[i] = predictions[i, 3]

sam_sub.to_csv("/content/e_us_best.csv", index = False)